In [130]:
import pandas as pd
import numpy as np
import random
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"



In [179]:
# ===== Конфиг =====
CONFIG = {
    "quarters_total": 20,
    "life_min": 12,
    "life_max": 16,
    "stages": {
        "q1": {
            "range": (1, 1),
            "revenue_min": 50000,
            "revenue_max": 150000,
            "cqgr_min": None,
            "cqgr_max": None,
            "margin_min": -2.5,
            "margin_max": -1.0,
        },
        "q24": {
            "range": (2, 4),
            "cqgr_min": 0.7,
            "cqgr_max": 1.0,
            "margin_min": -0.4,
            "margin_max": 0.2,
        },
        "q58": {
            "range": (5, 8),
            "cqgr_min": 0.4,
            "cqgr_max": 0.6,
            "margin_min": 0.4,
            "margin_max": 0.5,
        },
        "q912": {
            "range": (9, 12),
            "cqgr_min": 0.1,
            "cqgr_max": 0.2,
            "margin_min": 0.4,
            "margin_max": 0.5,
        },
        "late": {
            "range": (13, 20),
            "cqgr_min": 0.01,
            "cqgr_max": 0.09,
            "margin_min": 0.5,
            "margin_max": 0.6,
        }
    }
}
# ===== Функция =====
def simulate_startup_track_v2(config=CONFIG):
    q_total = config["quarters_total"]
    lifetime = random.randint(config["life_min"], config["life_max"])
    mrr = [None] * q_total
    current = None

    # Проходим по стадиям
    for stage, params in config["stages"].items():
        q_start, q_end = params["range"]

        # Q1 — инициализация revenue
        if stage == "q1":
            current = random.uniform(params["revenue_min"], params["revenue_max"])
            margin = random.uniform(params["margin_min"], params["margin_max"])
            mrr[q_start - 1] = {
                "revenue": current,
                "net": current * margin
            }
            continue

        # Для остальных стадий
        cqgr = random.uniform(params["cqgr_min"], params["cqgr_max"])
        for i in range(q_start - 1, min(q_end, lifetime)):
            current *= (1 + cqgr)
            margin = random.uniform(params["margin_min"], params["margin_max"])
            mrr[i] = {
                "revenue": current,
                "net": current * margin
            }

    return mrr

def generate_data_v2(quarters_total=20, startups_per_quarter=5, config=CONFIG):
    rows = []
    startup_id = 0

    for q in range(quarters_total):
        for _ in range(startups_per_quarter):
            startup_id += 1
            track = simulate_startup_track_v2(config=config)

            # сдвигаем трек стартапа к кварталу инвестиций
            shifted_track = [None] * q + track[:quarters_total - q]

            rows.append({
                "startup": f"Startup {startup_id}",
                "track": shifted_track
            })

    return rows

def generate_data_df(quarters_total=20, startups_per_quarter=5, config=CONFIG):
    """
    Генерация данных и возврат двух DataFrame:
    - df_revenue: значения revenue по стартапам и кварталам
    - df_net: значения net по стартапам и кварталам
    """
    startups = generate_data_v2(
        quarters_total=quarters_total,
        startups_per_quarter=startups_per_quarter,
        config=config
    )

    cols = [f"Q{i+1}" for i in range(quarters_total)]

    revenue_rows = []
    net_rows = []
    names = []

    for s in startups:
        names.append(s["startup"])
        rev_track = []
        net_track = []
        for val in s["track"]:
            if val is None:
                rev_track.append(None)
                net_track.append(None)
            else:
                rev_track.append(val["revenue"])
                net_track.append(val["net"])
        revenue_rows.append(rev_track)
        net_rows.append(net_track)

    df_revenue = pd.DataFrame(revenue_rows, columns=cols, index=names)
    df_net = pd.DataFrame(net_rows, columns=cols, index=names)

    return df_revenue, df_net

def apply_stage_mask_by_year(
    revenue_df: pd.DataFrame,
    net_df: pd.DataFrame,
    startups_per_quarter: int = 5,
    quarters_per_year: int = 4,
    cum_shares=None,      # (после Q1, после Q4, полный трек)
    cuts=(1, 4, None),           # сколько кварталов жизни оставить: 1, 4, None=без обрезки
    random_state=None
):
    """
    Маскирует треки стартапов по стадиям, равномерно по годам запуска (блоки по 4 квартала).

    - shares: доли в каждой годовой корзине (сумма ≈ 1.0)
      0 -> оставляем только 1 квартал жизни (после этого NaN)
      1 -> оставляем 4 квартала жизни (после этого NaN)
      2 -> полный трек
    - cuts: соответствующие "длины жизни" в кварталах (относительно старта)
      None = не обрезать

    Требования к входу:
      revenue_df, net_df — строки = стартапы, колонки = глобальные кварталы Q1..Qn,
      ряды уже сдвинуты (до старта — NaN).
    """

    assert revenue_df.shape == net_df.shape, "revenue_df и net_df должны быть одинаковой формы"
    n_rows, n_cols = revenue_df.shape
    cols = list(revenue_df.columns)

    rng = np.random.default_rng(random_state)


    # --- Преобразуем cum_shares → shares ---
    if cum_shares is not None:
        if len(cum_shares) != 2:
            raise ValueError("cum_shares должно быть длины 2: (p2, p3)")
        p2, p3 = cum_shares
        shares = (1 - p2, p2 - p3, p3)

    if shares is None:
        raise ValueError("Нужно задать либо shares, либо cum_shares")

    # Находим стартовый глобальный квартал каждого стартапа (первая НЕ NaN ячейка)
    mask_valid = revenue_df.notna().values
    start_pos = mask_valid.argmax(axis=1)  # индекс колонки первого True
    # Если у строки все NaN, argmax вернёт 0 — проверим и исключим такие случаи
    all_nan_rows = (~mask_valid).all(axis=1)
    if all_nan_rows.any():
        raise ValueError("Обнаружены строки без данных (все NaN). Проверь генерацию данных.")

    # Год запуска (0-базный) по глобальному кварталу старта
    start_year = start_pos // quarters_per_year

    # Подготовим копии для маскирования
    rev_out = revenue_df.copy()
    net_out = net_df.copy()

    # Группируем по годам запуска и применяем 50/30/20
    df_index = np.arange(n_rows)
    for yr in np.unique(start_year):
        idxs = df_index[start_year == yr]
        if len(idxs) == 0:
            continue

        # Перемешаем индексы внутри года
        shuffled = idxs.copy()
        rng.shuffle(shuffled)

        # Считаем численности по долям
        n = len(shuffled)
        n_after_q1 = int(round(n * (shares[0])))  # ~50%
        n_after_q4 = int(round(n * (shares[1])))  # ~30%
        # оставшиеся — полный трек
        n_full = max(0, n - n_after_q1 - n_after_q4)

        # Разбиваем
        group_after_q1 = shuffled[:n_after_q1]
        group_after_q4 = shuffled[n_after_q1:n_after_q1 + n_after_q4]
        group_full     = shuffled[n_after_q1 + n_after_q4:]

        # Функция маскирования "после k кварталов жизни"
        def _mask_after_k(row_idx: int, keep_k: int | None):
            s = start_pos[row_idx]  # глобальный индекс старта
            if keep_k is None:
                return  # полный трек, ничего не делаем
            cut_from = s + keep_k  # начиная ОТСЮДА ставим NaN
            if cut_from < n_cols:
                rev_out.iloc[row_idx, cut_from:] = np.nan
                net_out.iloc[row_idx, cut_from:] = np.nan

        # Применяем:
        for r in group_after_q1:
            _mask_after_k(r, cuts[0])  # оставить 1 квартал жизни
        for r in group_after_q4:
            _mask_after_k(r, cuts[1])  # оставить 4 квартала жизни
        for r in group_full:
            _mask_after_k(r, cuts[2])  # None -> без обрезки

        # (Опционально можно убедиться, что в каждом годе примерно 50/30/20)
        # print(f"year {yr}: n={n} -> 1Q:{len(group_after_q1)}, 4Q:{len(group_after_q4)}, full:{len(group_full)}")

    return rev_out, net_out

def simulate(n=1, quarters_total=20, startups_per_quarter=5, config=CONFIG):
    """
    Run n simulations.

    Возвращает два массива NumPy формы (n, quarters_total):
      - revenue_sims: суммы revenue по кварталам
      - net_sims: суммы net по кварталам
    """
    revenue_results = np.zeros((n, quarters_total))
    net_results = np.zeros((n, quarters_total))

    for sim in range(n):
        # Генерируем данные стартапов
        df_rev, df_net = generate_data_df(
            quarters_total=quarters_total,
            startups_per_quarter=startups_per_quarter,
            config=config
        )
        df_rev_masked, df_net_masked = apply_stage_mask_by_year( df_rev, df_net, startups_per_quarter=5, quarters_per_year=4, shares=(0.5, 0.3, 0.2),  cuts=(1, 4, None), random_state=42)
        # Складываем по всем стартапам => получаем ряд суммарного ревенью/нетто
        revenue_results[sim, :] = df_rev_masked.sum(axis=0, skipna=True).values
        net_results[sim, :] = df_net_masked.sum(axis=0, skipna=True).values

    # ✅ Возвращаем результаты
    return revenue_results, net_results
        
def plot_boxplots(rev_sims, net_sims):
    quarters_total = rev_sims.shape[1]

    # ===== Revenue =====
    df_rev = pd.DataFrame(rev_sims, columns=[f"Q{i+1}" for i in range(quarters_total)])
    df_rev_long = df_rev.melt(var_name="Quarter", value_name="Revenue")

    fig_rev = px.box(
        df_rev_long,
        x="Quarter",
        y="Revenue",
        title="Box plot по кварталам (Revenue)"
    )
    fig_rev.show()

    # ===== Net =====
    df_net = pd.DataFrame(net_sims, columns=[f"Q{i+1}" for i in range(quarters_total)])
    df_net_long = df_net.melt(var_name="Quarter", value_name="Net")

    fig_net = px.box(
        df_net_long,
        x="Quarter",
        y="Net",
        title="Box plot по кварталам (Net)"
    )
    fig_net.show()

In [ ]:
df_rev, df_net = generate_data_df(quarters_total=20, startups_per_quarter=5, config=CONFIG)


In [291]:
df_rev, df_net = generate_data_df(
            quarters_total=20,
            startups_per_quarter=5,
            config=CONFIG
        )
df_rev_masked, df_net_masked = apply_stage_mask_by_year( df_rev, df_net, startups_per_quarter=5, quarters_per_year=4, cum_shares=(0.5, 0.2),  cuts=(1, 4, None), random_state=None)

# Складываем по всем стартапам => получаем ряд суммарного ревенью/нетто
df_rev_masked[:20]

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
Startup 1,136880.425385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 2,112749.591179,197391.024166,345573.017285,604995.646483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 3,90770.611082,173566.332106,331883.538975,634608.579359,1.014548e+06,1.621958e+06,2.593022e+06,4.145463e+06,4.895133e+06,5.780375e+06,6.825705e+06,8.060073e+06,8.197600e+06,8.337475e+06,8.479736e+06,8.624424e+06,NaN,NaN,NaN,NaN
Startup 4,141284.311385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 5,88280.209157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 6,NaN,109525.219305,192373.075394,337889.304136,5.934780e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 7,NaN,54024.578392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 8,NaN,115142.497406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 9,NaN,129121.885058,247272.530542,473534.787174,9.068342e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 10,NaN,103108.099371,181367.632580,319026.520210,5.611692e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
df_net_masked.head(50)

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
Startup 1,-148293.550689,-25776.678041,-57537.867905,12235.382498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 2,-146736.982370,-6924.880904,-49989.271938,-78337.220385,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 3,-165498.499681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 4,-187962.593330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 5,-202273.577687,-26453.825404,36227.733662,-239490.065326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 6,NaN,-126127.596606,-65964.580521,-42559.188111,-107388.304597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 7,NaN,-200238.379285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 8,NaN,-235477.255513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 9,NaN,-195506.720290,-10048.801287,12117.352213,-113305.492392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 10,NaN,-92970.755145,-8569.238181,30495.462216,-79038.161977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# получили два DF из твоей функции генерации
df_rev, df_net = generate_data_df(quarters_total=20, startups_per_quarter=5, config=CONFIG)

# применили стадию отбора “равномерно по годам”
df_rev_masked, df_net_masked = apply_stage_mask_by_year(
    df_rev, df_net,
    startups_per_quarter=5,       # как у тебя в генерации
    quarters_per_year=4,
    shares=(0.5, 0.3, 0.2),       # 50% -> 1 квартал, 30% -> 4 квартала, 20% -> полный трек
    cuts=(1, 4, None),
    random_state=42
)


print(df_rev.sum().sum() / 1000000, df_net.sum().sum() / 1000000, df_rev_masked.sum().sum() / 1000000, df_net_masked.sum().sum() / 1000000)

2008.3340125789744 859.7168611659123 528.1737898696127 199.0636832007725


In [97]:
df_rev_masked


,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20
Startup 1,66466.128547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 2,121706.332661,207520.902807,353842.927974,603335.933796,906798.615361,1.362895e+06,2.048397e+06,3.078689e+06,3.570704e+06,4.141350e+06,4.803193e+06,5.570807e+06,5.738228e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 3,93968.148244,165852.832115,292728.572762,516662.972943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 4,114145.916523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Startup 5,79877.076424,144954.031309,263050.078113,477360.601635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Startup 96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90630.924390
Startup 97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114732.514622
Startup 98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103593.488756
Startup 99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80938.645953


In [116]:
rev_sims, net_sims = simulate(n=1000, quarters_total=20, startups_per_quarter=5, config=CONFIG)
plot_boxplots(rev_sims, net_sims)

In [111]:
net_sims[0].sum()

np.float64(157950774.31517524)

In [120]:
def investment_summary(strategy, quarters_total):
    startups_total = strategy["startups_per_quarter"] * quarters_total

    # Стадия 1
    stage1_count = startups_total
    stage1_total = stage1_count * strategy["stage1_check"]

    # Стадия 2
    stage2_count = int(stage1_count * strategy["survival_stage2"] / 100)
    stage2_total = stage2_count * strategy["stage2_check"]

    # Стадия 3
    stage3_count = int(stage2_count * strategy["survival_stage3"] / 100)
    stage3_total = stage3_count * strategy["stage3_check"]

    total_investment = stage1_total + stage2_total + stage3_total

    return {
        "Stage 1": {"startups": stage1_count, "investment": stage1_total},
        "Stage 2": {"startups": stage2_count, "investment": stage2_total},
        "Stage 3": {"startups": stage3_count, "investment": stage3_total},
        "Total Investment": total_investment
    }

# Пример использования
strategyInputs = {
    "startups_per_quarter": 5,
    "stage1_check": 100_000,
    "survival_stage2": 50,   # %
    "stage2_check": 500_000,
    "survival_stage3": 40,   # %
    "stage3_check": 2_000_000,
}

quarters_total = 20

investment_summary(strategyInputs, quarters_total)


{'Stage 1': {'startups': 100, 'investment': 10000000},
 'Stage 2': {'startups': 50, 'investment': 25000000},
 'Stage 3': {'startups': 20, 'investment': 40000000},
 'Total Investment': 75000000}

In [208]:
data = simulate_startup_track_v2()

In [285]:
import plotly.graph_objects as go
data = simulate_startup_track_v2()

# Извлекаем значения (None остаются None, Plotly сам сделает разрыв)
quarters = list(range(1, len(data) + 1))
revenue = [d['revenue'] / 1_000_000 if d else None for d in data]  # делим на млн
net = [d['net'] / 1_000_000 if d else None for d in data]  

# Создаем график
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=quarters, y=revenue,
    mode='lines+markers',
    name='Revenue',
    line=dict(color='blue'),
    hovertemplate='Quarter: %{x}<br>Revenue: %{y:.2f}M<extra></extra>',
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=quarters, y=net,
    mode='lines+markers',
    name='Net',
    line=dict(color='green'),
    hovertemplate='Quarter: %{x}<br>Net: %{y:.2f}M<extra></extra>',
    showlegend=False
))

# Находим последнюю точку, где есть данные
last_rev_idx = max(i for i, v in enumerate(revenue) if v is not None)
last_net_idx = max(i for i, v in enumerate(net) if v is not None)

# Добавляем подписи к линиям
fig.add_annotation(
    x=quarters[last_rev_idx], y=revenue[last_rev_idx],
    text="Revenue", showarrow=False, font=dict(color="blue", size=14), xanchor="left", yanchor="middle", xshift=20
)

fig.add_annotation(
    x=quarters[last_net_idx], y=net[last_net_idx],
    text="Net", showarrow=False, font=dict(color="green", size=14), xanchor="left", yanchor="middle", xshift=20
)

# Настройка оформления
# Настройка оформления
fig.update_layout(
    xaxis_title="Quarter",
    yaxis_title="Revenue / NET, M USD$",
    template="plotly_white",
    xaxis=dict(tickmode="array", tickvals=quarters, ticktext=quarters)
)

fig.show()